In [1]:
!pip install gdown


!gdown --id 13QE0lgBUsx5R7efKnIgeM7MJxdFnRwZi --output model.zip


/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=13QE0lgBUsx5R7efKnIgeM7MJxdFnRwZi
From (redirected): https://drive.google.com/uc?id=13QE0lgBUsx5R7efKnIgeM7MJxdFnRwZi&confirm=t&uuid=7f04fb81-a08b-470a-86d3-e2dd31e039bf
To: /content/model.zip
100% 158M/158M [00:04<00:00, 35.4MB/s]


In [2]:
!pip install -q unsloth datasets transformers
!pip uninstall -y unsloth && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install xformers

from unsloth import FastLanguageModel
from datasets import load_dataset
from transformers import TrainingArguments
from trl import SFTTrainer
import torch
import pandas as pd


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.9/163.9 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.9/310.9 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
!pip install -U bitsandbytes


In [5]:
import os
import torch
from torch.utils.data import DataLoader
from datasets import load_dataset
from transformers import TrainingArguments
from unsloth import FastLanguageModel
from trl import SFTTrainer

os.environ["WANDB_MODE"] = "disabled"
os.environ["WANDB_DISABLED"] = "true"

seed = 3407

device = "cuda" if torch.cuda.is_available() else "cpu"
dtype = torch.bfloat16 if device == "cuda" else None

dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp")

# Split dataset into training and validation sets
train_val_split = dataset['train'].train_test_split(test_size=0.1, seed=seed)
train_dataset = train_val_split['train'].select(range(0, 65000))
val_dataset = train_val_split['test'].select(range(20000, 20500))

Generating train split:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [8]:
import zipfile
with zipfile.ZipFile("model.zip", "r") as zip_ref:
    zip_ref.extractall("model_directory")

In [9]:
model_directory = "model_directory"

from unsloth import FastLanguageModel
from transformers import AutoTokenizer

model = FastLanguageModel.from_pretrained(model_directory)
tokenizer = AutoTokenizer.from_pretrained(model_directory)


==((====))==  Unsloth 2024.11.7: Fast Llama patching. Transformers = 4.46.2.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2024.11.7 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [12]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model, tokenizer = FastLanguageModel.from_pretrained(model_directory)

model = model.to(device)

model.eval()


==((====))==  Unsloth 2024.11.7: Fast Llama patching. Transformers = 4.46.2.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lor

In [13]:
tokenizer.padding_side = "left"


In [14]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import pandas as pd

FastLanguageModel.for_inference(model)
validation_dataset = dataset['train'].select(range(500000, 501000))
ground_truth = validation_dataset['is_correct']

#Prompt
prompt = """You are the best mathematician and have to determine if an answer to a given maths question is correct. Respond with 'True' if correct, otherwise 'False'.

### Question:
{}

### Answer:
{}

### Explanation:
{}

### Output:
{}"""

predictions = []
questions = []
answers = []
predicted_outputs = []

# Inference Loop
for i in range(len(validation_dataset)):
    question = validation_dataset['question'][i]
    answer = validation_dataset['answer'][i]
    solution = validation_dataset['solution'][i]
    is_correct_actual = ground_truth[i]

    input_prompt = prompt.format(question, answer, solution, "")

    # Tokenize and run inference
    inputs = tokenizer([input_prompt], return_tensors="pt").to(device)
    input_token_len = inputs['input_ids'].shape[1]
    outputs = model.generate(**inputs, max_new_tokens=1, use_cache=True)

    # Decode
    response = tokenizer.decode(outputs[0][input_token_len:], skip_special_tokens=True).strip()

    if "True" in response:
        is_correct_pred = True
    elif "False" in response:
        is_correct_pred = False
    else:
        is_correct_pred = False

    predictions.append(is_correct_pred)
    questions.append(question)
    answers.append(answer)
    predicted_outputs.append(response)

    print(f"Record {5000 + i}:")
    print(f"Question: {question}")
    print(f"Answer: {answer}")
    print(f"Predicted Output: {response}")
    print(f"Predicted is_correct: {is_correct_pred}")
    print(f"Actual is_correct: {is_correct_actual}")
    print("=" * 50)

accuracy = accuracy_score(ground_truth, predictions)
print(f"\nValidation Accuracy: {accuracy * 100:.2f}%")
print("\nConfusion Matrix:")
print(confusion_matrix(ground_truth, predictions))
print("\nClassification Report:")
print(classification_report(ground_truth, predictions, target_names=["False", "True"]))


Streaming output truncated to the last 5000 lines.
Predicted is_correct: True
Actual is_correct: True
Record 5387:
Question: What is $\frac{~\frac{2}{5}~}{\frac{3}{7}}$?
Answer: 14/15
Predicted Output: True
Predicted is_correct: True
Actual is_correct: True
Record 5388:
Question: The graph of the parabola $x = 2y^2 - 6y + 3$ has an $x$-intercept $(a,0)$ and two $y$-intercepts $(0,b)$ and $(0,c)$.  Find $a + b + c$.
Answer: \frac{3}{2}
Predicted Output: False
Predicted is_correct: False
Actual is_correct: False
Record 5389:
Question: Find the area of the triangle with vertices $(6,5,3),$ $(3,3,1),$ and $(15,11,9).$
Answer: 0
Predicted Output: False
Predicted is_correct: False
Actual is_correct: True
Record 5390:
Question: In rectangle $ABCD$, $P$ is a point on $BC$ so that $\angle APD=90^{\circ}$. $TS$ is perpendicular to $BC$ with $BP=PT$, as shown.  $PD$ intersects $TS$ at $Q$.  Point $R$ is on $CD$ such that $RA$ passes through $Q$.  In $\triangle PQA$, $PA=20$, $AQ=25$ and $QP=15$. 